<a href="https://colab.research.google.com/github/Kevinlo937/cord-cutting-predicated/blob/main/topMSO_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Load the required packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#Plot styling
import seaborn as sns; # sns.set()  # for plot styling
%matplotlib inline

In [ ]:
ds_test=pd.read_csv('/content/drive/Othercomputers/我的筆記型電腦/ExpertBook/2025/用戶資料集/PredictData/寬頻維修資料明細表_大屯_Q1_caret_delimited.csv', sep='^')

In [ ]:
dataset=pd.read_csv('/content/cleaned_dataset2.csv', sep='^')

In [ ]:
ds_test.head()

,系統台,派工單號,客戶編號,派工類別,受理日期,完工日期,申告原因,故障原因,故障原因(次),工程人員,產品包,派工備註
0,大屯,T2501000008,1167310,維修,2025-01-01 08:00:00,2025-01-01 09:15:48,雙向網路問題,"用戶端問題_IP分享(集線)器,無線AP問題",NaN,張宜勤,寬頻服務-台哥大好速成雙,"(系統紀錄)用戶目前的cmts為10.0.2.27,ISP出口為速博,去電0966-2319..."
1,大屯,T2412005387,97476,維修,2024-12-24 08:00:00,2025-01-01 10:27:19,雙向網路問題,tinp_CMTS問題,NaN,林文章,寬頻服務-飆網-豪華餐,"(系統紀錄)用戶目前的cmts為10.0.2.50,ISP出口為速博,上午維修 電0912-..."
2,大屯,T2412005169,901684,維修,2024-12-23 12:00:00,2025-01-01 10:28:31,雙向網路問題,RF線路問題_改良線路,NaN,林文章,寬頻服務,"(系統紀錄)用戶目前的cmts為10.0.2.50,ISP出口為速博,12/23下午 去電0..."
3,大屯,T2501000033,1009664,維修,2025-01-01 12:00:00,2025-01-01 11:30:50,雙向網路問題,其他,NaN,張苃銨,E-PON 寬頻服務-統簽方案(學舍),"(系統紀錄),ISP出口為E-PON速博,1/1 房客王先生0930-043626 ..."
4,大屯,T2412007161,1170176,維修,2025-01-01 08:00:00,2025-01-01 11:31:12,雙向網路問題,Epon實體問題_ONU變電器故障,NaN,鄭浡絃,FTTH-EPON 台哥大好速成雙,"(系統紀錄),ISP出口為E-PON速博,ONU非上線 1/1上午維修 近11點左右比較有空..."


In [ ]:
ds_doService = ds_test[['客戶編號', '受理日期', '完工日期']].copy()

In [ ]:
ds_doService.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1803 entries, 0 to 1802
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   客戶編號    1803 non-null   int64 
 1   受理日期    1803 non-null   object
 2   完工日期    1803 non-null   object
dtypes: int64(1), object(2)
memory usage: 42.4+ KB


In [ ]:
# Calculate the time difference in hours
ds_doService['等待時間'] = (pd.to_datetime(ds_doService['完工日期'], format='%Y-%m-%d %H:%M:%S', errors='coerce').astype('int64') // 10**9 - pd.to_datetime(ds_doService['受理日期'], format='%Y-%m-%d %H:%M:%S').astype('int64') // 10**9) / 3600
# Convert '等待時間' to days
ds_doService['等待天數'] = ds_doService['等待時間'] / 24  # Divide by 24 to convert hours to days
ds_doService['等待天數'] = ds_doService['等待天數'].round()

In [ ]:
ds_doService.head()

,客戶編號,受理日期,完工日期,等待時間,等待天數
0,1167310,2025-01-01 08:00:00,2025-01-01 09:15:48,1.263333,0.0
1,97476,2024-12-24 08:00:00,2025-01-01 10:27:19,194.455278,8.0
2,901684,2024-12-23 12:00:00,2025-01-01 10:28:31,214.475278,9.0
3,1009664,2025-01-01 12:00:00,2025-01-01 11:30:50,-0.486111,-0.0
4,1170176,2025-01-01 08:00:00,2025-01-01 11:31:12,3.520000,0.0


In [ ]:
# 針對 ds_doService['等待天數'] 的值做分布統計

# 1. 統計每個唯一值的出現次數
waiting_days_counts = ds_doService['等待天數'].value_counts()

# 2. 顯示統計結果
print(waiting_days_counts)

等待天數
 0.0     1595
-1.0       40
 4.0       27
 1.0       24
 7.0       22
 3.0       19
 2.0       11
 6.0       11
 8.0       10
 9.0        9
 10.0       7
 11.0       5
 5.0        3
 12.0       3
 14.0       3
 16.0       2
-2.0        2
-6.0        2
 15.0       2
-5.0        1
 40.0       1
-3.0        1
-13.0       1
 13.0       1
 17.0       1
Name: count, dtype: int64


In [ ]:
ds_doService.rename(columns={'客戶編號': '客編'}, inplace=True)

In [ ]:
ds_doService.head()

,客編,受理日期,完工日期,等待時間,等待天數
0,1167310,2025-01-01 08:00:00,2025-01-01 09:15:48,1.263333,0.0
1,97476,2024-12-24 08:00:00,2025-01-01 10:27:19,194.455278,8.0
2,901684,2024-12-23 12:00:00,2025-01-01 10:28:31,214.475278,9.0
3,1009664,2025-01-01 12:00:00,2025-01-01 11:30:50,-0.486111,-0.0
4,1170176,2025-01-01 08:00:00,2025-01-01 11:31:12,3.520000,0.0


In [ ]:
ds_doService.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1803 entries, 0 to 1802
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   客編      1803 non-null   int64  
 1   受理日期    1803 non-null   object 
 2   完工日期    1803 non-null   object 
 3   等待時間    1803 non-null   float64
 4   等待天數    1803 non-null   float64
dtypes: float64(2), int64(1), object(2)
memory usage: 70.6+ KB


In [ ]:
# ds_doService 為工單資料集，針對同一 '客編' 存在多筆工單紀錄時，比較其工單日期，以時間最末的工單日期為基礎，分別計算工單日期與其相距 30天內、60天內、90天內、以及90天以上的工單數量，並隨著相關編號存入 serviceDay_diff 資料集
# import pandas as pd

def calculate_service_day_diff(ds_doService):
    """
    Calculates the number of service orders within different time ranges for each '客編',
    accumulating counts based on where time_diffs fall within intervals.

    Args:
        ds_doService (pd.DataFrame): The service order dataset.

    Returns:
        pd.DataFrame: A DataFrame with '客編' and service order counts within time ranges.
    """
    # Convert '受理日期' to datetime objects
    ds_doService['受理日期'] = pd.to_datetime(ds_doService['受理日期'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

    # Group by '相關編號' and get the latest '工單日期' for each group
    latest_dates = ds_doService.groupby('客編')['受理日期'].max()

    # Create an empty dictionary to store the results
    service_day_diff = {}

    # Iterate through each unique '相關編號'
    for customer_id, latest_date in latest_dates.items():
        # Get all service orders for the current customer
        customer_orders = ds_doService[ds_doService['客編'] == customer_id]

        # Calculate the time difference between each order date and the latest date
        time_diffs = (latest_date - customer_orders['受理日期']).dt.days

        # Initialize counts for each range
        within_30_days = 0
        within_60_days = 0
        within_90_days = 0
        over_90_days = 0

        # Accumulate counts based on where time_diffs fall
        for diff in time_diffs:
            if diff <= 30:
                within_30_days += 1
            elif diff <= 60:
                within_60_days += 1
            elif diff <= 90:
                within_90_days += 1
            else:
                over_90_days += 1

        # Store the counts in the dictionary
        service_day_diff[customer_id] = {
            '受理日期': latest_date,
            '30天內': within_30_days,
            '60天內': within_60_days,
            '90天內': within_90_days,
            '90天以上': over_90_days
        }

    # Convert the dictionary to a DataFrame
    service_day_diff_df = pd.DataFrame.from_dict(service_day_diff, orient='index')
    service_day_diff_df.index.name = '客編'

    return service_day_diff_df

In [ ]:
# Assuming ds_doService is your DataFrame:
serviceDay_diff = calculate_service_day_diff(ds_doService)

In [ ]:
serviceDay_diff.head()

,受理日期,30天內,60天內,90天內,90天以上
客編,,,,,
1466,2025-01-05 15:30:00,1,0,0,0
3644,2025-02-24 18:30:00,1,0,0,0
3866,2025-03-25 12:00:00,1,0,0,0
5446,2025-01-09 18:30:00,1,0,0,0
6198,2025-02-17 12:00:00,1,0,0,0


In [ ]:
# ds_doService 為工單資料集，計算同一 '相關編號' 所對應 '等待天數'，計算'等待天數'的平均值，並隨同 '相關編號' 存入 aveWaittingDay
# Group data by '相關編號' and calculate the mean of '等待天數'
aveWaittingDay = ds_doService.groupby('客編')['等待天數'].mean()

# Reset the index to make '相關編號' a regular column
aveWaittingDay = aveWaittingDay.reset_index()

# Rename the column (optional) for better readability
aveWaittingDay = aveWaittingDay.rename(columns={'等待天數': '平均等待天數'})

# Display the result
print(aveWaittingDay)

           客編  平均等待天數
0        1466     4.0
1        3644     0.0
2        3866     0.0
3        5446     0.0
4        6198     0.0
...       ...     ...
1560  1270615     0.0
1561  1270885     0.0
1562  1271201     0.0
1563  1271266     0.0
1564  1271648     0.0

[1565 rows x 2 columns]


In [ ]:
# 依 '相關編號' 合併 serviceDay_diff 、aveWaittingDay，並加入ds_doService中同一 相關編號所屬 客編、相關編號，存入 ds_doService_combo 資料集
# Merge serviceDay_diff and aveWaittingDay
merged_df = pd.merge(serviceDay_diff, aveWaittingDay, on='客編', how='left')

# Get 客編 and 使用狀態 from ds_doService
customer_info = ds_doService[['客編']].drop_duplicates(subset=['客編'])

# Merge with the previous result
ds_doService_combo = pd.merge(customer_info, merged_df, on='客編', how='left')

print(ds_doService_combo)

           客編                受理日期  30天內  60天內  90天內  90天以上  平均等待天數
0     1167310 2025-01-01 08:00:00     1     0     0      0     0.0
1       97476 2024-12-24 08:00:00     1     0     0      0     8.0
2      901684 2024-12-23 12:00:00     1     0     0      0     9.0
3     1009664 2025-01-01 12:00:00     1     0     0      0     0.0
4     1170176 2025-01-01 08:00:00     1     0     0      0     0.0
...       ...                 ...   ...   ...   ...    ...     ...
1560   844377 2025-03-31 08:00:00     1     0     0      0     0.0
1561  1122269 2025-03-29 08:00:00     1     0     0      0     2.0
1562  1069161 2025-03-28 18:30:00     1     0     0      0     3.0
1563   894257 2025-03-31 18:30:00     1     0     0      0     0.0
1564  1075354 2025-03-31 12:00:00     1     0     0      0     0.0

[1565 rows x 7 columns]


In [ ]:
ds_doService_combo.rename(columns={'受理日期': '工單日期'}, inplace=True)

In [ ]:
ds_doService_combo.head()

,客編,工單日期,30天內,60天內,90天內,90天以上,平均等待天數
0,1167310,2025-01-01 08:00:00,1,0,0,0,0.0
1,97476,2024-12-24 08:00:00,1,0,0,0,8.0
2,901684,2024-12-23 12:00:00,1,0,0,0,9.0
3,1009664,2025-01-01 12:00:00,1,0,0,0,0.0
4,1170176,2025-01-01 08:00:00,1,0,0,0,0.0


In [ ]:
ds_doService_combo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1565 entries, 0 to 1564
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   客編      1565 non-null   int64         
 1   工單日期    1565 non-null   datetime64[ns]
 2   30天內    1565 non-null   int64         
 3   60天內    1565 non-null   int64         
 4   90天內    1565 non-null   int64         
 5   90天以上   1565 non-null   int64         
 6   平均等待天數  1565 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int64(5)
memory usage: 85.7 KB


In [ ]:
# ds_doService_combo 跟 dataset 做 merge 時，以ds_doService_combo 的客編為基準，dataset 中有多筆相同客編時，僅合併其使用狀態為'使用中' 的第一筆

# Filter 'dataset' for rows where '使用狀態' is '使用中'
dataset_filtered = dataset[dataset['使用狀態'] == '使用中']

# Drop duplicates, keeping the first occurrence of each '客編'
dataset_filtered = dataset_filtered.drop_duplicates(subset=['客編'], keep='first')

# Merge 'ds_doService_combo' and the filtered 'dataset'
ds_doService_combo = pd.merge(ds_doService_combo, dataset_filtered, on=['客編'], how='left')


In [ ]:
ds_doService_combo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1565 entries, 0 to 1564
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   客編      1565 non-null   int64         
 1   工單日期    1565 non-null   datetime64[ns]
 2   30天內    1565 non-null   int64         
 3   60天內    1565 non-null   int64         
 4   90天內    1565 non-null   int64         
 5   90天以上   1565 non-null   int64         
 6   平均等待天數  1565 non-null   float64       
 7   產品名稱    1091 non-null   object        
 8   用戶種類    1091 non-null   object        
 9   相關編號    1091 non-null   float64       
 10  起日      1091 non-null   object        
 11  迄日      184 non-null    object        
 12  系統台     1091 non-null   object        
 13  地區      1091 non-null   object        
 14  繳別      1091 non-null   float64       
 15  使用狀態    1091 non-null   object        
dtypes: datetime64[ns](1), float64(3), int64(5), object(7)
memory usage: 195.8+ KB


In [ ]:
# 刪 column
ds_doService_combo.drop(['工單日期', '產品名稱', '用戶種類','相關編號','起日', '迄日', '系統台', '地區', '使用狀態'], axis=1, inplace=True)

In [ ]:
ds_doService_combo.head()

,客編,30天內,60天內,90天內,90天以上,平均等待天數,繳別
0,1167310,1,0,0,0,0.0,1.0
1,97476,1,0,0,0,8.0,12.0
2,901684,1,0,0,0,9.0,12.0
3,1009664,1,0,0,0,0.0,NaN
4,1170176,1,0,0,0,0.0,1.0


In [ ]:
ds_doService_combo.to_csv('ds_test_大屯2025Q1.csv', index=False, sep='^')

In [2]:
X_test = pd.read_csv('/content/ds_test_大屯2025Q1.csv', sep='^')

In [3]:
X_test.head()

,客編,30天內,60天內,90天內,90天以上,平均等待天數,繳別
0,1167310,1,0,0,0,0.0,1.0
1,97476,1,0,0,0,8.0,12.0
2,901684,1,0,0,0,9.0,12.0
3,1009664,1,0,0,0,0.0,NaN
4,1170176,1,0,0,0,0.0,1.0


In [4]:
import joblib

loaded_model = joblib.load('knn_model.pkl')

In [5]:
X_test.dropna(inplace=True)
customer_ids = X_test[['客編']] # Extract both columns
X_test.drop(['客編' ], axis=1, inplace=True)
predictions =  loaded_model.predict(X_test)

# Create a DataFrame
results_df = pd.DataFrame({'客編': customer_ids['客編'],
                           'Prediction': predictions})
results_df.Prediction.value_counts()

,count
Prediction,
0,1073
1,18


In [6]:
# 篩選 Prediction 為 1 的資料
filtered_results_df = results_df[results_df['Prediction'] == 1]

# 列印篩選後的資料
print(filtered_results_df)

           客編  Prediction
117    966495           1
139   1263640           1
189    890361           1
245   1262827           1
296   1243659           1
526   1255953           1
535     96044           1
590   1157487           1
636   1086285           1
677   1180580           1
788   1230584           1
813    707589           1
895    793268           1
900   1267332           1
957    881977           1
1071  1216408           1
1189   851255           1
1387   729825           1
